# ROBOFISH

### Imports

In [ ]:
import FISH2_functions

### Find machine address
Use the below function to find the USB address and serial number of the connected devices, one device at a time.  
If you know the identifier of the FTDI chip of your device use it as input.  
Otherwise follow the instructions of the function and unplug and plug your device to find the address.  
  
When the search is successful add the serial number (preferred) or the USB COM port (alternative) to the data file.  
Open the data file with the `ROBOFISH_user_program.py` and add the info to the `ROBOFISH_System_datafile.yaml`.

In [ ]:
FISH2_functions.find_address(identifier=None)

# Initiate system

In [1]:
import FISH2_functions

#Path to the  database that is used to keep track of the experiment data. 
#database is automatically generated by the "FISH2_user_program.py" and prints the path after runing, copy the path to this location 
db_path = 'FISH_database\FISH_System2_db.sqlite'

#System specific path to start_imaging_file. If not present make a file called start_imaging_file.txt with a single 0.
#Then paste the path here, and also use this path for the imaging program.
start_imaging_file_path = "C:\\Users\\BL\\Desktop\\ROBOFISH\\FISH_database\\start_imaging_file.txt"

#System specific path to where the microscope saves the images. The info files will be put here.
#Use double slashes "\\" but do not put the trailing slashes! Example "C:\\Folder\\subfolder"
imaging_output_folder = "G:\\To_Monod\\5colortest"
#imaging_output_folder = "G:\\To_Monod\\LBEXP20210830_EEL_Mouse_Olfactory_3"


F2 = FISH2_functions.FISH2(db_path, imaging_output_folder, start_imaging_file_path, system_name='ROBOFISH2')

System name: ROBOFISH2
Path to database: FISH_database\FISH_System2_db.sqlite
Path to imaging_output_folder: G:\To_Monod\5colortest
Path to start_imaging_file_path: C:\Users\BL\Desktop\ROBOFISH\FISH_database\start_imaging_file.txt


Module ThermoCube not found. Ignore if machine is not in use.
Module Oasis not found. Ignore if machine is not in use.
Module YoctoThermisto_FISH not found. Ignore if machine is not in use.
Logger created. path:  log_files/2021-Sep-07_12-56-16_ROBOFISH2.log
FISH_database\FISH_System2_db.sqlite



Press Enter if:
            * Flowcell(s) is placed in stage and connected.
            * Heating/cooling cables are connected.
            * All buffers are in the correct position and connected.
            * System is connected.
            * Valves are open.
            * Datafile is committed to the database using the user program.
            * Optional temperature sensor is connected.
 


User confirmed system is ready for operation.
Retrieving data from database.
Successful retrieved data from database.
Connecting and initiating hardware.
    Finding device addresses.


Warning! No USB port information or serial identification code present for: CavroXCalibur add either to the Fixed_USB_port or Machine_identification table using the user program. Ignore if machine has its own identification method.


SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)

# Log manual protocol

In [2]:
#Lars

#Capture slide production
ITO_cleaning_date = '2021-06-29'
capture_slide_prep_date = '2021-07-08'

#Cutting
cutting_date = '2021-07-08'
block_temp = '-14C'
knife_temp = '-12C'
cutting_angle = 2
cutting_thickness = 10

#EEL
sample_prep_date = '2021-08-30'
protK_vol_per_ml = 2
probe_stock_volume = 50
probe_stock_name = 'Mouse 448 production 1 with amine'
probe_production_date = '2021-06-23' #'2021-02-24'
hybridization_start_time = '2021-08-30 at 11:30'
hybridization_stop_time = '2021-09-01 at 10:45'

prot = f"""MANUAL PROTOCOL

CAPTURE SLIDE:
ITO slides cleaned on: {ITO_cleaning_date}
Diamond coatings ITO slides with dimentions 24x60mm and #1.5 thickness, and ITO resistivity of 30-60Ohms/square.
Place ITO slides in bleakerglass.
Wash with Acetone, Iso-propanol and MQ water for 20 minutes each while sonicating on max.
Store in fresh MQ water.

Capture slide prepared on: {capture_slide_prep_date}
Place ITO slided in rack and dry with nitrogen gas.
Coat ITO slides for 2 hours in 2% (3-Glycidyloxypropyl)trimethoxysilane in acetone under nitrogen atmosphere at RT.
Rince slides once with acetone and dry under nitrogen gas.
Find ITO slide with Ohm meter.
Bind polyT at 10uM in 1X Schott spotting buffer by placing 35ul of mixtrue on ITO and cover with Hybrisilip.
Incubate 1 hour at 25C.
Wash 5 times with SSC 2X.
Wash once with dH2O.
Add 60ul of 0.1% (w/v) Poly-D-Lysin for 30 min.
Wash 3 times with dH2O.
Add 100ul of 1:250 small and 1:250 large europium beads for 3 min.
Pipett off and dry to air. 
If cutting performed on different day, store slides under nitrogen atmosphere.

CUTTING:
Cutting date: {cutting_date}
Cut sections on cryostat and capture on capture slides.
Block temp: {block_temp}
Knife temp: {knife_temp}
Angle: {cutting_angle} degree
Thickness: {cutting_thickness} um
Sections stored at -80C if not directly used

RNA TRANSFER:
Sample preparation performed on {sample_prep_date}.

Label the nucei with 1:1000 Propidium Iodine in SSC 2X for 5 minutes.
Wash 5 times with SSC 2X.
Place coverslip with parafilm gasket on top of tissue.
Select imaging area and perform reference imaging with 40X objective.

Remove Coverslip, gasket and SSC2X.
Permeabilize the tissue 5 minutes with 0.1% (v/v) Triton X-100, 10mM DTT in 1X TBE.
Wash 5 times with 1X TBE.
perform electrophoresis 20 minutes at 1.5V (15V/cm) in 1X TBE, 1M Urea and 10mM DTT.
Incubate 5 minues in 6X SSC.
Wash twice with 2X SSC.
During next digestion pipet up and down to disloge the tissue.
Digest 2 times 10 minutes with {protK_vol_per_ml}ul/ml ProtK NEB in digestion buffer (1% SDS, 20mM TrisHCl pH7.4, 5ul/ml Superase RNAse inhibitor) at 30C.
Wash 4 times 5 minutes with 5% SDS in 2X SSC at 30C.
Wash 5 times with 2X SSC.

Fix 10 minutes with 4% PFA in 1X PBS.
Wash 5 times with 2X SSC.

Dry {probe_stock_volume}ul of {probe_stock_name} produced on {probe_production_date} with speed vac.
Resuspend in 20ul of 30% formamide hybridization mix, 1nM/probe.

30% Hybridization mix contains:
    2X SSC, 
    0.1 g/ml Dextran sulfate, 
    1mg/ml E. Coli tRNA, 
    2mM Ribonucleoside Vanadyl Complex, 
    200ug/ml Bovine Serum Albumin
    30% (v/v) formamide

Place on tissue and cover with Grace Biolabs Hybrislip.
Incubate at 38.5 C in humidified petridish (paper towel wetted with 2X SSC, covered by parafilm).
Start: {hybridization_start_time}

Stop: {hybridization_stop_time}
Mount in Optical Biosystems flow cell and connect to ROBOFISH system

BUFFERS:
Readout mix in 10% Hybridization mix:
    2X SSC, 
    0.1 g/ml Dextran sulfate, 
    1mg/ml E. Coli tRNA, 
    2mM Ribonucleoside Vanadyl Complex, 
    200ug/ml Bovine Serum Albumin
    10% (v/v) formamide
    50mM Fluorescent readout probes.

Wash buffer after encoding probes
    2X SSC,
    30% (v/v) formamide.
    
Wash buffer after readout probes
    2X SSC,
    20% (v/v) formamide.

Imaging buffer:
    2X SSC,
    2mM Trolox (6-hydroxy-2,5,7,8-tetramethylchroman-2-carboxylic acid)
    5mM 3,4-Dihydroxybenzoic acid
    20nM Protocatechuate 3,4-Dioxygenase from Pseudomonas sp.

Stripping buffer:
    2X SSC,
    50mM TCEP

"""
F2.L.logger.info(prot)
F2.L.logger.info('____')

MANUAL PROTOCOL

CAPTURE SLIDE:
ITO slides cleaned on: 2021-06-29
Diamond coatings ITO slides with dimentions 24x60mm and #1.5 thickness, and ITO resistivity of 30-60Ohms/square.
Place ITO slides in bleakerglass.
Wash with Acetone, Iso-propanol and MQ water for 20 minutes each while sonicating on max.
Store in fresh MQ water.

Capture slide prepared on: 2021-07-08
Place ITO slided in rack and dry with nitrogen gas.
Coat ITO slides for 2 hours in 2% (3-Glycidyloxypropyl)trimethoxysilane in acetone under nitrogen atmosphere at RT.
Rince slides once with acetone and dry under nitrogen gas.
Find ITO slide with Ohm meter.
Bind polyT at 10uM in 1X Schott spotting buffer by placing 35ul of mixtrue on ITO and cover with Hybrisilip.
Incubate 1 hour at 25C.
Wash 5 times with SSC 2X.
Wash once with dH2O.
Add 60ul of 0.1% (w/v) Poly-D-Lysin for 30 min.
Wash 3 times with dH2O.
Add 100ul of 1:250 small and 1:250 large europium beads for 3 min.
Pipett off and dry to air. 
If cutting performed on diff

In [ ]:
#Alejandro

#Capture slide production
ITO_cleaning_date = '2021-02-07'
capture_slide_prep_date = '2020-02-14'

#Cutting
cutting_date = '2021-02-17'
block_temp = '-14C'
knife_temp = '-12C'
cutting_angle = 0
cutting_thickness = 10

#EEL
sample_prep_date = '2021-02-20'
protK_vol_per_ml = 1
probe_stock_volume = 30
probe_stock_name = 'HE448 production 1'
probe_production_date = '2020-11-30'
hybridization_start_time = '2021-02-23 at 19:00'
hybridization_stop_time = '2021-02-26 at 15:30'

prot = f"""MANUAL PROTOCOL

CAPTURE SLIDE:
ITO slides cleaned on: {ITO_cleaning_date}
Diamond coatings ITO slides with dimentions 24x60mm and #1.5 thickness, and ITO resistivity of 30-60Ohms/square.
Place ITO slides in bleakerglass.
Wash with Acetone, Iso-propanol and MQ water for 20 minutes each while sonicating on max.
Store in fresh MQ water.

Capture slide prepared on: {capture_slide_prep_date}
Place ITO slided in rack and dry with nitrogen gas.
Coat ITO slides for 2 hours in 2% (3-Glycidyloxypropyl)trimethoxysilane in acetone under nitrogen atmosphere at RT.
Rince slides once with acetone and dry under nitrogen gas.
Find ITO slide with Ohm meter.
Bind polyT at 10uM in 1X Schott spotting buffer by placing 35ul of mixtrue on ITO and cover with Hybrisilip.
Incubate 1 hour at 25C.
Wash 5 times with SSC 2X.
Wash once with dH2O.
Add 60ul of 0.1% (w/v) Poly-D-Lysin for 30 min.
Wash 3 times with dH2O.
Add 100ul of 1:250 small and 1:250 large europium beads for 3 min.
Pipett off and dry to air. 
If cutting performed on different day, store slides under nitrogen atmosphere.

CUTTING:
Cutting date: {cutting_date}
Cut sections on cryostat and capture on capture slides.
Block temp: {block_temp}
Knife temp: {knife_temp}
Angle: {cutting_angle} degree
Thickness: {cutting_thickness} um
Sections stored at -80C if not directly used

RNA TRANSFER:
Sample preparation performed on {sample_prep_date}.

Label the nucei with 1:1000 Propidium Iodine in SSC 2X for 5 minutes.
Wash 5 times with SSC 2X.
Place coverslip with parafilm gasket on top of tissue.
Select imaging area and perform reference imaging with 40X objective.

Remove Coverslip, gasket and SSC2X.
Permeabilize the tissue 5 minutes with 0.1% (v/v) Triton X-100, 10mM DTT in 1X TBE.
Wash 5 times with 1X TBE.
perform electrophoresis 20 minutes at 1.5V (15V/cm) in 1X TBE, 1M Urea and 10mM DTT.
Incubate 5 minues in 6X SSC.
Wash twice with 2X SSC.
During next digestion pipet up and down to disloge the tissue.
Digest 2 times 10 minutes with {protK_vol_per_ml}ul/ml ProtK NEB in digestion buffer (1% SDS, 20mM TrisHCl pH7.4, 5ul/ml Superase RNAse inhibitor) at 30C.
Wash 4 times 5 minutes with 5% SDS in 2X SSC at 30C.
Wash 5 times with 2X SSC.

Fix 10 minutes with 4% PFA in 1X PBS.
Wash 5 times with 2X SSC.

Dry {probe_stock_volume}ul of {probe_stock_name} produced on {probe_production_date} with speed vac.
Resuspend in 20ul of 30% formamide hybridization mix, 1nM/probe.

30% Hybridization mix contains:
    2X SSC, 
    0.1 g/ml Dextran sulfate, 
    1mg/ml E. Coli tRNA, 
    2mM Ribonucleoside Vanadyl Complex, 
    200ug/ml Bovine Serum Albumin
    30% (v/v) formamide

Place on tissue and cover with Grace Biolabs Hybrislip.
Incubate at 38.5 C in humidified petridish (paper towel wetted with 2X SSC, covered by parafilm).
Start: {hybridization_start_time}

Stop: {hybridization_stop_time}
Mount in Optical Biosystems flow cell and connect to ROBOFISH system

BUFFERS:
Readout mix in 10% Hybridization mix:
    2X SSC, 
    0.1 g/ml Dextran sulfate, 
    1mg/ml E. Coli tRNA, 
    2mM Ribonucleoside Vanadyl Complex, 
    200ug/ml Bovine Serum Albumin
    10% (v/v) formamide
    50mM Fluorescent readout probes.

Wash buffer after encoding probes
    2X SSC,
    30% (v/v) formamide.
    
Wash buffer after readout probes
    2X SSC,
    20% (v/v) formamide.

Imaging buffer:
    2X SSC,
    2mM Trolox (6-hydroxy-2,5,7,8-tetramethylchroman-2-carboxylic acid)
    5mM 3,4-Dihydroxybenzoic acid
    20nM Protocatechuate 3,4-Dioxygenase from Pseudomonas sp.

Stripping buffer:
    2X SSC,
    50mM TCEP

"""
F2.L.logger.info(prot)
F2.L.logger.info('____')

# Individual functions

In [3]:
def EEL_washBufferWash(chamber, volume, cycles, incubation_time=0, buffer='WB'):
    """
    Stringency wash after hybridization. Including SSC2X wash afterwards.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of WB to use per wash cycle.
    `cycles`(int): Number of cycle to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    
    """
    F2.L.logger.info('Stingency wash {} start'.format(chamber))
    #Wash with WB
    F2.extractDispenseBuffer(buffer, volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer(buffer, volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)
    #Wash out WB
    for i in range(4):
        F2.extractDispenseRunningBuffer(1000, chamber)
    F2.L.logger.info('Stringency wash {}. Washed {} times with {}ul WB for {} minutes'.format(chamber, cycles, volume, incubation_time))
    
def EEL_fix(chamber):
    """
    Perform PFA fixation manually.
    Input:
    `chamber`(str): Chamber to fix. Like: 'Chamber1'.
    
    """
    #Manual Fixation
    long_message='Manually fix 10 minutes with 4% PFA in 1X PBS. Afterwards flush with SSC2X'
    F2.push(short_message='Fixation chamber{}'.format(chamber), long_message=long_message)
    print(long_message)
    #Set temperature
    F2.setTemp(F2.Parameters['Imaging_temperature'], chamber)
    F2.waitTemp(F2.Parameters['Imaging_temperature'], chamber, error=3, sd=0.02, verbose=False)
    input('Perform Fixation. Press enter when done with fixation...')
    F2.L.logger.info('Manually fixed 10 minutes with 4% PFA in 1X PBS.')
    F2.L.logger.info('Manually flushed with SSC2X.')
    #Set temperature
    F2.setTemp(F2.Parameters['Staining_temperature'], chamber)
    F2.extractDispenseRunningBuffer(1000, chamber)
    F2.waitTemp(F2.Parameters['Staining_temperature'], chamber, error=3, sd=0.02, verbose=False)
    
def EEL_imagingBuffer(chamber, volume, imaging_temp):
    """
    Inject imaging buffer into the chamber and set chamber to imaging temperature.
    Input:
    `chamber`(str): Chamber to inject. Like: 'Chamber1'.
    `volume`(int): Volume of imaging buffer to inject.
    `imaging_temp`(float): Temperature to perform the imaging at.
    
    """
    F2.L.logger.info('Imaging buffer injecting into {}'.format(chamber))
    F2.setTemp(imaging_temp, chamber)
    F2.extractDispenseBuffer('IB', volume, chamber, padding=True, same_buffer_padding=False)
    F2.waitTemp(imaging_temp, chamber, error=3, sd=0.02, verbose=False)
    F2.L.logger.info('Imaging buffer. Injected {}ul of imaging buffer into {}, temperature set to {}C'.format(volume, chamber, imaging_temp))

def EEL_stripping(chamber, volume, cycles, stripping_temp, incubation_time=0, wash_cycles= 5):
    """
    Stripping off hybridized probes. Including SSC2X wash before (to wash out IB)
    and after.
    Input:
    `chamber`(str): Chamber to wash. Like: 'Chamber1'.
    `volume`(int): Volume of SB to use per wash cycle.
    `cycles`(int): Number of cycles to wash.
    `incubation_time`(int): Incubation time in minutes per wash cycle.
    `stripping_temp`(int): Temperature used for hybridization/washes in Celcius.
    `wash_cycles`(int): Number of 1000ul washes after the striping. Default=5
    
    """
    F2.L.logger.info('Stripping {} start'.format(chamber))
    F2.setTemp(stripping_temp, chamber)
    #Wash out imaging buffer
    for i in range(4):
        F2.extractDispenseRunningBuffer(1000, chamber, padding=False)
    F2.waitTemp(stripping_temp, chamber, error=3, sd=0.02, verbose=False)
    F2.extractDispenseBuffer('SB', volume, chamber, padding=True, same_buffer_padding=True, double_volume=True, m=incubation_time)
    F2.resetReservoir(50, update_buffer=True)
    for i in range(cycles-1):
        F2.extractDispenseBuffer('SB', volume, chamber, padding=False, m=incubation_time)
        F2.resetReservoir(50, update_buffer=True)   
    #Wash out SB
    for i in range(wash_cycles):
        F2.extractDispenseRunningBuffer(2000, chamber, padding=False)
    F2.L.logger.info('Stripping. Washed {}, {} times with {}ul SB for {} minutes'.format(chamber, cycles, volume, incubation_time))
    
def EEL_hybmix(chamber, cycle, hybridization_temp, incubation_time, indirect=None,):
    """
    Hybridization.
    Input:
    `chamber`(str): Chamber to hybridize. Like: 'Chamber1'.
    `cycle`(int): Current staining cycle.
    `hybridization_temp`(float): Hybridization temperature.
    `incubation_time`(int): Hybridization time in HOURS.
    `indirect`(str): Amplifier code. 
    
    """
    #Set temperature
    F2.setTemp(hybridization_temp, chamber)
    F2.waitTemp(hybridization_temp, chamber, error=3, sd=0.02, verbose=False)
    
    F2.L.logger.info('Hybridization {}, cycle {} start dispense'.format(chamber, cycle))
    #F2.extractDispenseHybmix(chamber, cycle, slow_speed=27, prehyb=False, steps=0, h=incubation_time, indirect=indirect) #slow speed: 2.5ml/min
    F2.extractDispenseHybmix(chamber, cycle, slow_speed=32, prehyb=False, steps=0, h=incubation_time, indirect=indirect) #slow speed: 1.25ml/min
    F2.L.logger.info('Hybridization {}, cycle {} finished'.format(chamber, cycle))

# Combined functions

In [4]:
def EEL_readout(chamber, cycle):
    """
    Perform the first round of the EEL detection
    
    """
    
    #Set temperature
    #F2.setTemp(47, chamber)
    #F2.extractDispenseRunningBuffer(1000, chamber)
    #F2.waitTemp(47, chamber, error=3, sd=0.02, verbose=False)
    
    #Washing
    #EEL_washBufferWash(chamber, 600, 4, 15, buffer='WB')
    #F2.L.logger.info('Changed 30% formamide wash buffer to 20% formamide wash buffer. ')
    
    #PFA fization
    #EEL_fix(chamber)
    
    #F2.extractDispenseRunningBuffer(1000, 'Chamber1')
    
    #Hybridization
    #hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    #hyb_time = F2.Parameters[hyb_time_code]
    #EEL_hybmix(chamber, cycle, F2.Parameters['Staining_temperature'], hyb_time)


    #Washing
    #EEL_washBufferWash(chamber, 600, 3, 2)
    
    #Imaging    
    #EEL_imagingBuffer(chamber, 600, F2.Parameters['Imaging_temperature'])
    F2.push(short_message='{} Ready for Imaging round: {}'.format(chamber, cycle))
    
    
    
def EEL_readout_repeat(chamber, cycle):
    """
    Perform the repeat round of the EEL detection
    
    """
    #stripping
    EEL_stripping(chamber, 600, 2, F2.Parameters['Stripping_temperature'], 10, wash_cycles=6)
    
    #if cycle == 17:
    #    F2.L.logger.info('Performing stripping with 65% formamide')
    #    F2.L.logger.info('Manually inject 65% formamide in 2X SSC. 2x10min at 25C.')
    #    F2.setTemp(25, chamber)
    #    F2.waitTemp(25, chamber, error=3, sd=0.02, verbose=False)   
    #    input('Press enter when done with stripping...')
    #    input('Sure...')
        
    #    for i in range(5):
    #        F2.extractDispenseRunningBuffer(1000, chamber)

    #Wash out IB if stripping is not performed seperately
    #for i in range(4):
    #    F2.extractDispenseRunningBuffer(1000, chamber)

    #Hybridization
    hyb_time_code = 'Hyb_time_{}_A'.format(chamber[-1])
    hyb_time = F2.Parameters[hyb_time_code]
    EEL_hybmix(chamber, cycle, F2.Parameters['Staining_temperature'], hyb_time)

    #Washing
    EEL_washBufferWash(chamber, 600, 3, 2)   
        
    #Imaging
    EEL_imagingBuffer(chamber, 600, F2.Parameters['Imaging_temperature'])
    F2.push(short_message='{} Ready for Imaging round: {}'.format(chamber, cycle))

In [5]:
F2.L.logger.info('Made a mistake in the code so the program crassed. See previous log file for Round 1.')

Made a mistake in the code so the program crassed. See previous log file for Round 1.


# Scheduler

In [7]:
F2.scheduler(EEL_readout, EEL_readout_repeat, remove_experiment=False, log_info_file=True)

Start new experiment, start with chamber 1.
_____
STARTING LBEXP20210830_EEL_Mouse_Olfactory_3, CYCLE: 1
Experiment configuration file created: G:\To_Monod\LBEXP20210830_EEL_Mouse_Olfactory_3\LBEXP20210830_EEL_Mouse_Olfactory_3_config.yaml



Cycle parameters:
round_code: C1H01
experiment_name: LBEXP20210830_EEL_Mouse_Olfactory_3
Description: Mouse olfactory bulb 448 genes
Protocols_io: https://www.protocols.io/edit/eel-t92er8e
chamber: chamber1
Machine: ROBOFISH2
Operator: lars
Timestamp_robofish: 2021-09-01 13-27-09
hybridization_fname: Unknown-at-dict-generation-time
hybridization_number: 1
Hyb_time_A: 0.16
Hyb_time_B: None
Hyb_time_C: None
Hybmix_volume: 500
Imaging_temperature: 20.0
Fluidic_Program: None
Readout_temperature: 22.0
Staining_temperature: 37.0
Start_date: 20210830
Target_cycles: 16
Species: Mus Musculus
Sample: B283 and B281
Strain: CD1
Age: P24
Tissue: Brain
Orientation: Corronal
RegionImaged: Olfactory bulb
SectionID: 2
Position: None
Experiment_type: eel-barcoded
Chemistry: EELV2_all
Probe_FASTA: {'DAPI': 'None', 'Atto425': 'None', 'FITC': 'None', 'Cy3': 'None', 'TxRed': 'None', 'Cy5': 'M448.fasta', 'Cy7': 'None', 'Europium': 'None'}
Barcode: True
Barcode_length: 16
Codebooks: {'DAPI': 'None', 'Atto425

Imaging of other chamber finished, waited 0 minutes. (This time could be used to extend the hybridization with 0.0 hours.)

Waiting for imaging to finish...


Imaging Experiment: LBEXP20210830_EEL_Mouse_Olfactory_3 Cycle: 1 done.

 
_____
STARTING LBEXP20210830_EEL_Mouse_Olfactory_3, CYCLE: 2
Stripping Chamber1 start


Imaging of other chamber finished, waited 38 minutes. (This time could be used to extend the hybridization with 0.63 hours.)

Expected finish time of cycle 2 in Chamber1: 2021-09-01 15:59:33.166103
Expected finish time for full experiment in Chamber1: 2021-09-02 09:46:20.814786



...

    TC720 set to 22.0 degree Celsius.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Chamber1 within range of target temperature 22.0C, allowed error 3C. Reached in 6 seconds after starting the waitTemp() function.
    Dispensed 1200ul of SB to Chamber1 with speed 20, padding=True, same_buffer_padding=True, double_volume=True
    Dispensed 600ul of SB to Chamber1 with speed 20, padding=False, same_buffer_padding=False, double_volume=False
    Dispensed 2000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 2000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 2000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 2000ul of SSC2X to Chamber1, with speed 20, padding=0.
    Dispensed 2000ul of SSC2X to Chamber1, with 

!!WARNING!! To prevent reading/writing to the same file, Notepad++.exe will be closed in:
< 0 > secondsClosed down Notepad++.exe
You will be notified when Notepad++.exe can be used again
Removed Hybmix code from P20
Notepad++.exe can be used again
Hybridization start time: 02-09-2021 19:53:25, Done at: 02-09-2021 20:03:01


    Washed port P20 5 times with RunningBuffer: SSC2X.
Hybridization Chamber1, cycle 16 finished
Stingency wash Chamber1 start
    Dispensed 1200ul of WB to Chamber1 with speed 20, padding=True, same_buffer_padding=True, double_volume=True
    Dispensed 600ul of WB to Chamber1 with speed 20, padding=False, same_buffer_padding=False, double_volume=False
    Dispensed 600ul of WB to Chamber1 with speed 20, padding=False, same_buffer_padding=False, double_volume=False
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=897.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=897.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=897.
    Dispensed 1000ul of SSC2X to Chamber1, with speed 20, padding=897.
Stringency wash Chamber1. Washed 3 times with 600ul WB for 2 minutes
Imaging buffer injecting into Chamber1
    TC720 set to 20.0 degree Celsius.
    Dispensed 600ul of IB to Chamber1 with speed 20, padding=True, same_buffer_padding=False, double

Imaging of other chamber finished, waited 0 minutes. (This time could be used to extend the hybridization with 0.0 hours.)
LBEXP20210830_EEL_Mouse_Olfactory_3 Finished
Full experiment completed. 
                        Ready to start a new experiment. Starting secure sleep



KeyboardInterrupt: 